In [1]:
from transformers import TFBertForSequenceClassification, BertTokenizer
import tensorflow as tf
from data_preparation import Example, convert_examples_to_tf_dataset

In [2]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
model.bert.trainable = False
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertForSequenceClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier', 'dropout_37']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy()
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [4]:
dataset = convert_examples_to_tf_dataset([(Example("hello there", 1)), 
                                          (Example("dog", 0)), 
                                          (Example("hello", 1)),
                                          (Example("dog here", 0))
                                          ], tokenizer, max_length=5)
dataset = dataset.shuffle(100).batch(2).repeat(5)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (

In [23]:
list(iter(dataset))[0]

({'input_ids': <tf.Tensor: shape=(2, 5), dtype=int32, numpy=
  array([[  101, 17835,   102,     0,     0],
         [  101, 17835, 19353,   102,     0]])>,
  'attention_mask': <tf.Tensor: shape=(2, 5), dtype=int32, numpy=
  array([[1, 1, 1, 0, 0],
         [1, 1, 1, 1, 0]])>,
  'token_type_ids': <tf.Tensor: shape=(2, 5), dtype=int32, numpy=
  array([[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]])>},
 <tf.Tensor: shape=(2,), dtype=int64, numpy=array([0, 0], dtype=int64)>)

In [25]:
model.fit(dataset, epochs=5, steps_per_epoch=2)

Epoch 1/5
2/2 [==============================] - 0s 19ms/step - loss: 0.6597 - sparse_categorical_accuracy: 0.7500
Epoch 2/5
2/2 [==============================] - 0s 16ms/step - loss: 0.7152 - sparse_categorical_accuracy: 0.5000
Epoch 3/5
2/2 [==============================] - 0s 18ms/step - loss: 0.7186 - sparse_categorical_accuracy: 0.2500
Epoch 4/5
2/2 [==============================] - 0s 18ms/step - loss: 0.7090 - sparse_categorical_accuracy: 0.2500
Epoch 5/5
2/2 [==============================] - 0s 16ms/step - loss: 0.6904 - sparse_categorical_accuracy: 0.5000


In [26]:
test = convert_examples_to_tf_dataset([(Example("hello friend", 1))], tokenizer, max_length=5)
test = test.shuffle(100).batch(1)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [27]:
model.evaluate(test)

1/1 [==============================] - 0s 179ms/step - loss: 0.7530 - sparse_categorical_accuracy: 0.0000e+00


[0.7529698014259338, 0.0]